# Using the `gsw` toolbox to compute density

This tutorial shows how to use the [`gsw` toolbox](https://teos-10.github.io/GSW-Python/) (the Gibbs SeaWater Oceanographic Toolbox of TEOS-10) within Parcels to compute density from temperature and salinity fields. The `gsw` toolbox can be installed via `conda install gsw`.

First, load the necessary libraries and the data:

In [ ]:
import numpy as np
import xarray as xr

import parcels

# Load the CopernicusMarine data in the Agulhas region from the example_datasets
example_dataset_folder = parcels.download_example_dataset(
    "CopernicusMarine_data_for_Argo_tutorial"
)

ds = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")

# TODO check how we can get good performance without loading full dataset in memory
ds.load()  # load the dataset into memory

fieldset = parcels.FieldSet.from_copernicusmarine(ds)

Now, define a custom Particle class that includes temperature, salinity, and density as variables, and create a ParticleSet with one particle at a known location:

In [ ]:
GSWParticle = parcels.Particle.add_variable(
    [
        parcels.Variable("temp", dtype=np.float32, initial=np.nan),
        parcels.Variable("salt", dtype=np.float32, initial=np.nan),
        parcels.Variable("density", dtype=np.float32, initial=np.nan),
    ]
)

# Initiate one Argo float in the Agulhas Current
pset = parcels.ParticleSet(
    fieldset=fieldset,
    pclass=GSWParticle,
    lon=[32],
    lat=[-31],
    depth=[200],
)

Now (as the core part of this tutorial) define a custom kernel that uses the `gsw` toolbox to compute density from temperature and salinity:

In [ ]:
def ParcelsGSW(particles, fieldset):
    import gsw

    particles.temp = fieldset.thetao[particles]
    particles.salt = fieldset.so[particles]
    pressure = gsw.p_from_z(-particles.depth, particles.lat)
    particles.density = gsw.density.rho(particles.salt, particles.temp, pressure)

Finally, run the `ParcelsGSW` Kernel for one timestep and check (for Continuous Integration purposes) that the computed density is as expected:

In [ ]:
pset.execute(ParcelsGSW, runtime=np.timedelta64(1, "s"), dt=np.timedelta64(1, "s"))

np.testing.assert_allclose(pset.density, [1026.8281], rtol=1e-5)

print(
    f"Temperature: {pset.temp[0]:.2f}, Salinity: {pset.salt[0]:.2f}, Density: {pset.density[0]:.2f}"
)